In [1]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0
Thu Nov 10 16:46:17 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |      

Mount Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Benchmark 1: lavaMD

##lavaMD - CUDA
Necessary change in makefile:
- Set `-arch=sm_75` (because the Tesla T4 has the Turing architecture: https://askubuntu.com/questions/1157589/nvcc-fatal-value-sm-20-is-not-defined-for-option-gpu-architecture)

To reduce warnings:
- Add `#include <sys/time.h>` to `./utils/timer.c`
- Add `#include <string.h>` and `#include <time.h>` to `./main.c`
- Replace all percentage signs '%' that should be actually printed in `printf()` commands with `%%` in `./kernel/kernel_gpu_cuda_wrapper.cu`

In [3]:
%cd '/content/drive/MyDrive/Colab Notebooks/Applied GPU Programming KTH/Assignment_2/rodinia_3.1/cuda/lavaMD'
!ls

/content/drive/MyDrive/Colab Notebooks/Applied GPU Programming KTH/Assignment_2/rodinia_3.1/cuda/lavaMD
kernel	lavaMD	main.c	main.h	main.o	makefile  README  result.txt  run  util


In [4]:
!make clean
!make KERNEL_DIM="-DRD_WG_SIZE_0=128"

rm	*.o \
	./kernel/*.o \
	./util/num/*.o \
	./util/timer/*.o \
	./util/device/*.o \
	lavaMD
gcc	-DRD_WG_SIZE_0=128  main.c \
		-c \
		-o main.o \
		-O3
/usr/local/cuda/bin/nvcc -DRD_WG_SIZE_0=128						./kernel/kernel_gpu_cuda_wrapper.cu \
								-c \
								-o ./kernel/kernel_gpu_cuda_wrapper.o \
								-O3 \
								-arch sm_75
./kernel/kernel_gpu_cuda_wrapper.cu: In function ‘void kernel_gpu_cuda_wrapper(par_str, dim_str, box_str*, FOUR_VECTOR*, double*, FOUR_VECTOR*)’:
./kernel/kernel_gpu_cuda_wrapper.cu:66:23: warning: ‘cudaError_t cudaThreadSynchronize()’ is deprecated [-Wdeprecated-declarations]
  cudaThreadSynchronize();
                       ^
/usr/local/cuda/bin/../targets/x86_64-linux/include/cuda_runtime_api.h:1011:46: note: declared here
 extern __CUDA_DEPRECATED __host__ cudaError_t CUDARTAPI cudaThreadSynchronize(void);
                                              ^~~~~~~~~~~~~~~~~~~~~
./kernel/kernel_gpu_cuda_wrapper.cu:196:23: warning: ‘cudaError_t cudaThreadSynchr

In [5]:
!ls
!./lavaMD -boxes1d 10

kernel	lavaMD	main.c	main.h	main.o	makefile  README  result.txt  run  util
thread block size of kernel = 128 
Configuration used: boxes1d = 10
Time spent in different stages of GPU_CUDA KERNEL:
 0.300085008144 s, 59.149749755859 % : GPU: SET DEVICE / DRIVER INIT
 0.000372000010 s,  0.073324911296 % : GPU MEM: ALO
 0.001872000052 s,  0.368989855051 % : GPU MEM: COPY IN
 0.203749999404 s, 40.161155700684 % : GPU: KERNEL
 0.000761999981 s,  0.150197803974 % : GPU MEM: COPY OUT
 0.000490000006 s,  0.096583887935 % : GPU MEM: FRE
Total time:
0.507331013680 s


##lavaMD - OMP

To reduce warnings:
- Add `#include <sys/time.h>` to `./utils/timer.c`
- Add `#include <string.h>` and `#include <time.h>` to `./main.c`
- Replace all percentage signs '%' that should be actually printed in `printf()` commands with `%%` in `./kernel/kernel_cpu.c`

In [6]:
%cd '/content/drive/MyDrive/Colab Notebooks/Applied GPU Programming KTH/Assignment_2/rodinia_3.1/openmp/lavaMD'
!ls

/content/drive/MyDrive/Colab Notebooks/Applied GPU Programming KTH/Assignment_2/rodinia_3.1/openmp/lavaMD
kernel	lavaMD	main.c	main.h	main.o	makefile  README  run  util


In [7]:
!make clean
!make

rm	*.o \
	./kernel/*.o \
	./util/num/*.o \
	./util/timer/*.o \
                ./util/device/*.o \
	lavaMD
rm: cannot remove './util/device/*.o': No such file or directory
makefile:67: recipe for target 'clean' failed
make: *** [clean] Error 1
gcc	 main.c \
		-c \
		-o main.o \
		-O3
gcc				./kernel/kernel_cpu.c \
					-c \
					-o ./kernel/kernel_cpu.o \
					-O3 \
					-fopenmp
gcc			./util/num/num.c \
				-c \
				-o ./util/num/num.o \
				-O3
gcc				./util/timer/timer.c \
					-c \
					-o ./util/timer/timer.o \
					-O3
gcc	main.o \
		./kernel/kernel_cpu.o \
		./util/num/num.o \
		./util/timer/timer.o \
		-lm \
		-lgomp \
		-o lavaMD


In [8]:
!ls
!./lavaMD -cores 4 -boxes1d 10

kernel	lavaMD	main.c	main.h	main.o	makefile  README  run  util
Configuration used: cores = 4, boxes1d = 10
Time spent in different stages of CPU/MCPU KERNEL:
 0.000001000000 s,  0.000015313983 % : CPU/MCPU: VARIABLES
 0.000009000000 s,  0.000137825846 % : MCPU: SET DEVICE
 0.000000000000 s,  0.000000000000 % : CPU/MCPU: INPUTS
 6.529970169067 s, 99.999847412109 % : CPU/MCPU: KERNEL
Total time:
6.529980182648 s


# Benchmark 2: lud

##lud - CUDA

Necessary change in makefile:
- Set `-arch=sm_75` in `./cuda/Makefile` (because the Tesla T4 has the Turing architecture: https://askubuntu.com/questions/1157589/nvcc-fatal-value-sm-20-is-not-defined-for-option-gpu-architecture)

In [9]:
%cd '/content/drive/MyDrive/Colab Notebooks/Applied GPU Programming KTH/Assignment_2/rodinia_3.1/cuda/lud'
!ls

/content/drive/MyDrive/Colab Notebooks/Applied GPU Programming KTH/Assignment_2/rodinia_3.1/cuda/lud
base  common  cuda  Makefile  README  run  tools


In [10]:
!make clean
!make KERNEL_DIM="-DRD_WG_SIZE_0=64" # Change in makefile necessary: https://askubuntu.com/questions/1157589/nvcc-fatal-value-sm-20-is-not-defined-for-option-gpu-architecture

cd cuda; make clean
make[1]: Entering directory '/content/drive/MyDrive/Colab Notebooks/Applied GPU Programming KTH/Assignment_2/rodinia_3.1/cuda/lud/cuda'
rm -f lud_cuda ../common/common.o lud.o lud_kernel.o *.linkinfo
make[1]: Leaving directory '/content/drive/MyDrive/Colab Notebooks/Applied GPU Programming KTH/Assignment_2/rodinia_3.1/cuda/lud/cuda'
cd cuda; make
make[1]: Entering directory '/content/drive/MyDrive/Colab Notebooks/Applied GPU Programming KTH/Assignment_2/rodinia_3.1/cuda/lud/cuda'
nvcc -DRD_WG_SIZE_0=64 -I../common -O3 -use_fast_math -arch sm_75 -lm  -DGPU_TIMER  -o ../common/common.o -c ../common/common.c
../common/common.c: In function ‘create_matrix_from_file’:
../common/common.c:51:3: warning: ignoring return value of ‘fscanf’, declared with attribute warn_unused_result [-Wunused-result]
   fscanf(fp, "%d\n", &size);
   ^~~~~~~~~~~~~~~~~~~~~~~~~
../common/common.c:61:11: warning: ignoring return value of ‘fscanf’, declared with attribute warn_unused_result [-Wunu

In [11]:
!ls
!./cuda/lud_cuda -s 2048

base  common  cuda  Makefile  README  run  tools
WG size of kernel = 64 X 64
Generate input matrix internally, size =2048
Creating matrix internally size=2048
Time consumed(ms): 48.456000


##lud - OMP
Necesarry changes in `./omp/Makefile.offload`: <br />
`CC              := icc` -> `CC              := gcc` <br />
`CXX             := icc` -> `CXX             := gcc` <br />

In [12]:
%cd '/content/drive/MyDrive/Colab Notebooks/Applied GPU Programming KTH/Assignment_2/rodinia_3.1/openmp/lud'
!ls

/content/drive/MyDrive/Colab Notebooks/Applied GPU Programming KTH/Assignment_2/rodinia_3.1/openmp/lud
base  common  Makefile	omp  README  run  run_offload  tools


In [13]:
!make clean
!make

cd omp; make clean ; make -f Makefile.offload clean
make[1]: Entering directory '/content/drive/MyDrive/Colab Notebooks/Applied GPU Programming KTH/Assignment_2/rodinia_3.1/openmp/lud/omp'
rm --force lud_omp lud.o lud_omp.o ../common/common.o *~
make[1]: Leaving directory '/content/drive/MyDrive/Colab Notebooks/Applied GPU Programming KTH/Assignment_2/rodinia_3.1/openmp/lud/omp'
make[1]: Entering directory '/content/drive/MyDrive/Colab Notebooks/Applied GPU Programming KTH/Assignment_2/rodinia_3.1/openmp/lud/omp'
rm --force lud_omp_offload lud.o lud_omp.o ../common/common.o *~
make[1]: Leaving directory '/content/drive/MyDrive/Colab Notebooks/Applied GPU Programming KTH/Assignment_2/rodinia_3.1/openmp/lud/omp'
cd omp; make
make[1]: Entering directory '/content/drive/MyDrive/Colab Notebooks/Applied GPU Programming KTH/Assignment_2/rodinia_3.1/openmp/lud/omp'
gcc  -c -Wall -O3   -fopenmp -I../common lud.c -o lud.o
gcc  -c -Wall -O3   -fopenmp -I../common lud_omp.c -o lud_omp.o
gcc  -c -W

In [14]:
!ls
!./omp/lud_omp -s 2048

base  common  Makefile	omp  README  run  run_offload  tools
Generate input matrix internally, size =2048
Creating matrix internally size=2048
running OMP on host
Time consumed(ms): 740.013000


# Benchmark 3: myocyte

##myocyte - CUDA
To reduce warnings:
- Replace all percentage signs '%' that should be actually printed in `printf()` commands with `%%` in `./work.cu` and `./work_2.cu`

In [15]:
%cd '/content/drive/MyDrive/Colab Notebooks/Applied GPU Programming KTH/Assignment_2/rodinia_3.1/cuda/myocyte'
!ls

/content/drive/MyDrive/Colab Notebooks/Applied GPU Programming KTH/Assignment_2/rodinia_3.1/cuda/myocyte
define.c		    kernel_cam.cu    main.cu	  run
embedded_fehlberg_7_8_2.cu  kernel.cu	     main.o	  solver_2.cu
embedded_fehlberg_7_8.cu    kernel_ecc_2.cu  Makefile	  solver.cu
file.c			    kernel_ecc.cu    master.cu	  timer.c
kernel_2.cu		    kernel_fin_2.cu  myocyte.out  work_2.cu
kernel_cam_2.cu		    kernel_fin.cu    README	  work.cu


In [16]:
!make clean
!make KERNEL_DIM="-DRD_WG_SIZE_0=64" # Change in makefile necessary: https://askubuntu.com/questions/1157589/nvcc-fatal-value-sm-20-is-not-defined-for-option-gpu-architecture

rm *.o *.out output.txt
rm: cannot remove 'output.txt': No such file or directory
Makefile:43: recipe for target 'clean' failed
make: *** [clean] Error 1
nvcc	main.cu \
			-c -O3 -g
file.c(108): warning: result of call is not used

file.c(116): warning: result of call is not used

work.cu(126): warning: conversion from a string literal to "char *" is deprecated

work.cu(135): warning: conversion from a string literal to "char *" is deprecated

work_2.cu(171): warning: conversion from a string literal to "char *" is deprecated

work_2.cu(185): warning: conversion from a string literal to "char *" is deprecated

file.c(108): warning: result of call is not used

file.c(116): warning: result of call is not used

work.cu(126): warning: conversion from a string literal to "char *" is deprecated

work.cu(135): warning: conversion from a string literal to "char *" is deprecated

work_2.cu(171): warning: conversion from a string literal to "char *" is deprecated

work_2.cu(185): warning: conver

In [17]:
!ls
!./myocyte.out 1000 100 1

define.c		    kernel_cam.cu    main.cu	  run
embedded_fehlberg_7_8_2.cu  kernel.cu	     main.o	  solver_2.cu
embedded_fehlberg_7_8.cu    kernel_ecc_2.cu  Makefile	  solver.cu
file.c			    kernel_ecc.cu    master.cu	  timer.c
kernel_2.cu		    kernel_fin_2.cu  myocyte.out  work_2.cu
kernel_cam_2.cu		    kernel_fin.cu    README	  work.cu
Time spent in different stages of the application:
0.000001000000 s, 0.000009827563 % : SETUP VARIABLES
1.510107994080 s, 14.840682029724 % : ALLOCATE CPU MEMORY AND GPU MEMORY
1.729429960251 s, 16.996082305908 % : READ DATA FROM FILES, COPY TO GPU MEMORY
0.000036000001 s, 0.000353792275 % : RUN GPU KERNEL
6.933597087860 s, 68.140365600586 % : COPY GPU DATA TO CPU MEMORY
0.002289999975 s, 0.022505121306 % : FREE MEMORY
Total time:
10.175461769104 s


##myocyte - OMP
To reduce warnings:
- Replace all percentage signs '%' that should be actually printed in `printf()` commands with `%%` in `./main.c`

In [18]:
%cd '/content/drive/MyDrive/Colab Notebooks/Applied GPU Programming KTH/Assignment_2/rodinia_3.1/openmp/myocyte'
!ls

/content/drive/MyDrive/Colab Notebooks/Applied GPU Programming KTH/Assignment_2/rodinia_3.1/openmp/myocyte
cam.c	  embedded_fehlberg_7_8.c  main.c    master.c	  run
define.c  file.c		   main.o    myocyte.out  solver.c
ecc.c	  fin.c			   Makefile  README	  timer.c


In [19]:
!make clean
!make

rm *.o myocyte.out output.txt
rm: cannot remove 'output.txt': No such file or directory
Makefile:32: recipe for target 'clean' failed
make: *** [clean] Error 1
gcc	main.c \
		-c -O3 -fopenmp
main.c: In function ‘main’:
main.c:175:20: warning: format ‘%d’ expects argument of type ‘int’, but argument 2 has type ‘long int’ [-Wformat=]
    printf("ERROR: %d is the incorrect end of simulation interval, use numbers > 0\n", xmax);
                   ~^
                   %ld
main.c:185:20: warning: format ‘%d’ expects argument of type ‘int’, but argument 2 has type ‘long int’ [-Wformat=]
    printf("ERROR: %d is the incorrect number of instances of simulation, use numbers > 0\n", workload);
                   ~^
                   %ld
In file included from main.c:91:0:
file.c: In function ‘read’:
file.c:109:5: warning: ignoring return value of ‘fscanf’, declared with attribute warn_unused_result [-Wunused-result]
     fscanf(fid, "%f", &temp);
     ^~~~~~~~~~~~~~~~~~~~~~~~
file.c:117:5: warni

In [20]:
!ls
!./myocyte.out 1000 100 1 4

cam.c	  embedded_fehlberg_7_8.c  main.c    master.c	  run
define.c  file.c		   main.o    myocyte.out  solver.c
ecc.c	  fin.c			   Makefile  README	  timer.c
The file was not opened for reading
Time spent in different stages of the application:
0.000011000000 s, 0.000091592447 % : SETUP VARIABLES, READ COMMAND LINE ARGUMENTS
0.018454000354 s, 0.153658822179 % : ALLOCATE MEMORY
0.128046005964 s, 1.066185951233 % : READ DATA FROM FILES
11.850337028503 s, 98.672851562500 % : RUN COMPUTATION
0.012876000255 s, 0.107213124633 % : FREE MEMORY
Total time:
12.009723663330 s
